In [1]:
# Import necessary modules
import boto3
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time
import uuid

from botocore.config import Config

here = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(here, '../../'))
sys.path.insert(0, project_root)

from core.model.model_helpers import get_sim_inputs_from_hourly
from core.plotting.plotting import *

## Model Parameters

In [2]:
# Set ModelParams (time stepping)
dt               = 1.0
simulation_hours = 2900
total_time_steps = int(simulation_hours / dt)

DATA_BUCKET = "smartfarm-data"
DATA_KEY    = "hourly_prcp_rad_temp_iowa.csv"

context = {
    "dt": float(dt),
    "total_time_steps": int(total_time_steps),
    "simulation_hours": int(simulation_hours),
    "closed_form":      False,

    # S3 location of disturbances
    "data_bucket": DATA_BUCKET,
    "data_key":    DATA_KEY,
}

## Upload input disturbance data to s3

In [3]:
s3 = boto3.client("s3")

local_path  = "../../io/inputs/hourly_prcp_rad_temp_iowa.csv"  # where the file is on your laptop

s3.upload_file(local_path, DATA_BUCKET, DATA_KEY)
print("Uploaded to s3://%s/%s" % (DATA_BUCKET, DATA_KEY))

Uploaded to s3://smartfarm-data/hourly_prcp_rad_temp_iowa.csv


## Define the weight option combinations to sweep over

In [4]:
weight_irrigation_options                     = [0.01, 0.1, 1.0, 10]
weight_fertilizer_options                     = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0]
weight_fruit_biomass_options                  = [1.0, 10.0, 100.0, 1000.0]
weight_cumulative_average_water_options       = [1.0]
weight_cumulative_average_fertilizer_options  = [1.0]
weight_cumulative_average_temperature_options = [1.0]
weight_cumulative_average_radiation_options   = [1.0]

In [5]:
# Serialize all weight combinations to dict of lists
weights = []
for wI in weight_irrigation_options:
    for wF in weight_fertilizer_options:
        for wB in weight_fruit_biomass_options:
            for wCAW in weight_cumulative_average_water_options:
                for wCAF in weight_cumulative_average_fertilizer_options:
                    for wCAT in weight_cumulative_average_temperature_options:
                        for wCAR in weight_cumulative_average_radiation_options:
                            weights.append(
                                {
                                    "weight_irrigation":                     wI,
                                    "weight_fertilizer":                      wF,
                                    "weight_fruit_biomass":                   wB,
                                    "weight_cumulative_average_water":        wCAW,
                                    "weight_cumulative_average_fertilizer":   wCAF,
                                    "weight_cumulative_average_temperature":  wCAT,
                                    "weight_cumulative_average_radiation":    wCAR,
                                }
                            )

In [6]:
len(weights)

96

## Run each of the combinations as their own Lambda instance

In [7]:
# Fire batches in parallel
REGION         = "us-west-1"
FUNCTION_NAME  = "smartfarm-mpc-container"
BUCKET         = "smartfarm-data"

lambda_client = boto3.client(
    "lambda",
    region_name=REGION,
    config=Config(retries={"max_attempts": 10, "mode": "adaptive"})
)

s3 = boto3.client("s3", region_name=REGION)

# Your list of 96 weight dictionaries
num_runs = len(weights)

run_id = time.strftime("%Y%m%d_%H%M%S") + "_" + uuid.uuid4().hex[:8]
print("run_id =", run_id)

for idx, weights_dict in enumerate(weights):
    payload = {
        "run_id":       run_id,
        "idx":          idx,
        "weights_dict": weights_dict,
        "context":      context,
    }
    lambda_client.invoke(
        FunctionName=FUNCTION_NAME,
        InvocationType="Event",  # async
        Payload=json.dumps(payload).encode("utf-8"),
    )

run_id = 20251217_235028_70c1471a


In [9]:
resp = s3.list_objects_v2(Bucket=DATA_BUCKET, Prefix=f"runs/{run_id}/")

if "Contents" not in resp:
    print("No results found for that run_id yet.")
else:
    records = []
    for obj in resp["Contents"]:
        key = obj["Key"]
        # skip "directory" placeholders if any
        if not key.endswith(".json"):
            continue

        body = s3.get_object(Bucket=DATA_BUCKET, Key=key)["Body"].read()
        rec  = json.loads(body.decode("utf-8"))
        records.append(rec)

    df = pd.DataFrame(records).sort_values("idx").reset_index(drop=True)
    print(df.head())

No results found for that run_id yet.
